In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 2: Rock Paper Scissors ---</h2><p>The Elves begin to set up camp on the beach. To decide whose tent gets to be closest to the snack storage, a giant <a href="https://en.wikipedia.org/wiki/rock_paper_scissors" target="_blank">Rock Paper Scissors</a> tournament is already in progress.</p>
<p>Rock Paper Scissors is a game between two players. Each game contains many rounds; in each round, the players each simultaneously choose one of Rock, Paper, or Scissors using a hand shape. Then, a winner for that round is selected: Rock defeats Scissors, Scissors defeats Paper, and Paper defeats Rock. If both players choose the same shape, the round instead ends in a draw.</p>
<p>Appreciative of your help yesterday, one Elf gives you an <em>encrypted strategy guide</em> (your puzzle input) that they say will be sure to help you win. "The first column is what your opponent is going to play: <code>A</code> for Rock, <code>B</code> for Paper, and <code>C</code> for Scissors. The second column--" Suddenly, the Elf is called away to help with someone's tent.</p>
<p>The second column, <span title="Why do you keep guessing?!">you reason</span>, must be what you should play in response: <code>X</code> for Rock, <code>Y</code> for Paper, and <code>Z</code> for Scissors. Winning every time would be suspicious, so the responses must have been carefully chosen.</p>
<p>The winner of the whole tournament is the player with the highest score. Your <em>total score</em> is the sum of your scores for each round. The score for a single round is the score for the <em>shape you selected</em> (1 for Rock, 2 for Paper, and 3 for Scissors) plus the score for the <em>outcome of the round</em> (0 if you lost, 3 if the round was a draw, and 6 if you won).</p>
<p>Since you can't be sure if the Elf is trying to help you or trick you, you should calculate the score you would get if you were to follow the strategy guide.</p>
<p>For example, suppose you were given the following strategy guide:</p>
<pre><code>A Y
B X
C Z
</code></pre>
<p>This strategy guide predicts and recommends the following:</p>
<ul>
<li>In the first round, your opponent will choose Rock (<code>A</code>), and you should choose Paper (<code>Y</code>). This ends in a win for you with a score of <em>8</em> (2 because you chose Paper + 6 because you won).</li>
<li>In the second round, your opponent will choose Paper (<code>B</code>), and you should choose Rock (<code>X</code>). This ends in a loss for you with a score of <em>1</em> (1 + 0).</li>
<li>The third round is a draw with both players choosing Scissors, giving you a score of 3 + 3 = <em>6</em>.</li>
</ul>
<p>In this example, if you were to follow the strategy guide, you would get a total score of <code><em>15</em></code> (8 + 1 + 6).</p>
<p><em>What would your total score be if everything goes exactly according to your strategy guide?</em></p>
</article>


In [2]:
def rock_paper_scissors(s: str) -> int:
    score = {"A": 1, "B": 2, "C": 3, "X": 1, "Y": 2, "Z": 3}
    strategie = [[score[c] for c in p.strip().split()] for p in s.strip().splitlines()]

    total_score = 0

    for opponent, me in strategie:
        total_score += me

        match me, opponent:
            case 1, 3:
                total_score += 6
            case 3, 2:
                total_score += 6
            case 2, 1:
                total_score += 6
            case x, y if x == y:
                total_score += 3

    return total_score


rock_paper_scissors_tests = [
    {
        "name": "Example",
        "s": """
            A Y
            B X
            C Z
        """,
        "expected": 15,
    },
]


@test(tests=rock_paper_scissors_tests)
def rock_paper_scissors_test(s: str) -> int:
    return rock_paper_scissors(s)


Test Example passed, for rock_paper_scissors_test.
Success


In [3]:
with open("../input/day2.txt") as f:
    puzzle = f.read()

print(f"Part 1: {rock_paper_scissors(puzzle)}")

Part 1: 13052


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>13052</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The Elf finishes helping with the tent and sneaks back over to you. "Anyway, the second column says how the round needs to end: <code>X</code> means you need to lose, <code>Y</code> means you need to end the round in a draw, and <code>Z</code> means you need to win. Good luck!"</p>
<p>The total score is still calculated in the same way, but now you need to figure out what shape to choose so the round ends as indicated. The example above now goes like this:</p>
<ul>
<li>In the first round, your opponent will choose Rock (<code>A</code>), and you need the round to end in a draw (<code>Y</code>), so you also choose Rock. This gives you a score of 1 + 3 = <em>4</em>.</li>
<li>In the second round, your opponent will choose Paper (<code>B</code>), and you choose Rock so you lose (<code>X</code>) with a score of 1 + 0 = <em>1</em>.</li>
<li>In the third round, you will defeat your opponent's Scissors with Rock for a score of 1 + 6 = <em>7</em>.</li>
</ul>
<p>Now that you're correctly decrypting the ultra top secret strategy guide, you would get a total score of <code><em>12</em></code>.</p>
<p>Following the Elf's instructions for the second column, <em>what would your total score be if everything goes exactly according to your strategy guide?</em></p>
</article>


In [4]:
def rock_paper_scissors_part2(s: str) -> int:
    score = {"A": 1, "B": 2, "C": 3, "X": "lose", "Y": "draw", "Z": "win"}
    chose = {
        "lose": {1: 3, 2: 1, 3: 2},
        "draw": {1: 1, 2: 2, 3: 3},
        "win": {1: 2, 2: 3, 3: 1},
    }
    strategie = [[score[c] for c in p.strip().split()] for p in s.strip().splitlines()]

    total_score = 0

    for opponent, strategie in strategie:
        if strategie == "win":
            total_score += 6
        elif strategie == "draw":
            total_score += 3
        total_score += chose[strategie][opponent]

    return total_score


rock_paper_scissors_part2_tests = [
    {
        "name": "Example",
        "s": """
            A Y
            B X
            C Z
        """,
        "expected": 12,
    },
]


@test(tests=rock_paper_scissors_part2_tests)
def rock_paper_scissors_part2_test(s: str) -> int:
    return rock_paper_scissors_part2(s)


Test Example passed, for rock_paper_scissors_part2_test.
Success


<link href="style.css" rel="stylesheet"></link>


In [6]:
print(f"Part 2: {rock_paper_scissors_part2(puzzle)}")

Part 2: 13693


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>13693</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
